In [79]:
import librosa as lb
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
location = "D:/dnn_data/"
duration = 4
sampling_rate = 44100
#3*44100 = 259,40
#259/7 = 37
total_length = duration * sampling_rate
# 데이터 추출(chroma, mfcc)
def featureExtract(file_location, select, n):
    #샘플링된 주파수 로드
    data, sr = lb.core.load(file_location, sr=sampling_rate, mono=True, res_type="kaiser_fast") 
    # 파일이 일정이상 커지면 그 뒤 값은 모두 랜덤
    if len(data)> total_length:
        max_offset = len(data) - total_length
        offset = np.random.randint(max_offset)
        data = data[offset:(total_length+offset)]
    else:
        if total_length > len(data):
            max_offset = total_length - len(data)
            offset = np.random.randint(max_offset)
        else:
            offset = 0
        data = np.pad(data, (offset, total_length - len(data) - offset), "constant")
    if select == 0:   
        data = lb.feature.chroma_stft(data, sr=sampling_rate, n_chroma=n)
        data = np.reshape(data,10350)
    elif select ==1:
        data = lb.feature.mfcc(data, sr=sampling_rate, n_mfcc=n)
        # z변환
        m =  np.mean(data, axis=0)
        std= np.std(data, axis=0)
        data = (data-m)/std
        data = np.reshape(data,10350)
    else:
        data = lb.feature.melspectrogram(data, sr=sampling_rate)
    return data

#데이터 로드
train = pd.read_csv(location+"audio_train.csv")

#인덱스 설정(label에 각각 숫자부여)
label_index = pd.DataFrame(columns=("labeled_num","label"))
label_index["labeled_num"] = range(len(train["label"].unique()))
label_index["label"] = train['label'].unique()
train=pd.merge(train, label_index, how="left")

# 숫자와 파일이름 빼고 제거
train = train.drop(['label','manually_verified'],axis=1)
trainFname = train.values[:,0]

#chroma_data 초기화
rows=len(train)
#pd.Series(featureExtract(location+'audio_train/8343332e.wav')).append('1')
#chroma_data = np.zeros((len(train),10361))
mfcc_data = np.zeros((len(train), 10351))
mfcc_data.shape

(9473, 10351)

In [72]:
# 데이터 전체 chroma-stft
i=0
for fname in trainFname:
    labeled_num=train.loc[train['fname']==fname,"labeled_num"]
    #indexed_extract=np.hstack([labeled_num,featureExtract(location+"/audio_train/"+fname)])
    #chroma_data[i] = indexed_extract
    indexed_extract=np.hstack([labeled_num,featureExtract(location+"/audio_train/"+fname,1,30)])
    mfcc_data[i] = indexed_extract
    indexed_extract.shape
    i+=1
    if i%9==0:
        #print(i,'번째:',chroma_data[i])
        print(i,'번째:',mfcc_data[i])
    #길이:10350 = 30*345(4s)


9 번째: [ 8.         -5.1731099  -5.16735512 ...  0.15797524  0.18276593
  0.22772622]
18 번째: [14.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
27 번째: [19.         -5.35758864 -5.35178129 ...  0.12344084  0.03907517
  0.02908468]
36 번째: [25.         -4.15394327 -4.10577874 ...  0.13864078  0.1859154
  0.26116081]
45 번째: [25.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
54 번째: [19.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
63 번째: [25.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
72 번째: [ 9.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
81 번째: [31.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
90 번째: [33.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
99 번째: [18.         -5.3177029  -5.31877587 ...  0.17385433  0.13541805
  0.19253211]
108 번째: [25.         -4.65059156 -4.72224161 ...  0.3611

846 번째: [19.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
855 번째: [ 3.70000000e+01 -3.96836739e+00 -3.78707290e+00 ...  1.33016578e-01
  8.64678208e-02 -1.05803894e-03]
864 번째: [10.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
873 번째: [20.         -5.33553354 -5.33749673 ...  0.18443036  0.12747573
  0.12641002]
882 번째: [ 4.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
891 번째: [19.         -5.00809578 -5.0027803  ...  0.07998945  0.09809984
  0.06211844]
900 번째: [13.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
909 번째: [ 1.60000000e+01 -4.39889556e+00 -4.57859228e+00 ...  1.34404144e-01
  8.10641430e-02  8.82283592e-03]
918 번째: [10.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
927 번째: [ 9.         -5.38516481 -5.38516481 ...  0.18569534  0.18569534
  0.18569534]
936 번째: [25.         -5.36178143 -5.32997003 ...  0.10441597  0.1453165
  0.137738

2376 번째: [0. 0. 0. ... 0. 0. 0.]
2385 번째: [0. 0. 0. ... 0. 0. 0.]
2394 번째: [0. 0. 0. ... 0. 0. 0.]
2403 번째: [0. 0. 0. ... 0. 0. 0.]
2412 번째: [0. 0. 0. ... 0. 0. 0.]
2421 번째: [0. 0. 0. ... 0. 0. 0.]
2430 번째: [0. 0. 0. ... 0. 0. 0.]
2439 번째: [0. 0. 0. ... 0. 0. 0.]
2448 번째: [0. 0. 0. ... 0. 0. 0.]
2457 번째: [0. 0. 0. ... 0. 0. 0.]
2466 번째: [0. 0. 0. ... 0. 0. 0.]
2475 번째: [0. 0. 0. ... 0. 0. 0.]
2484 번째: [0. 0. 0. ... 0. 0. 0.]
2493 번째: [0. 0. 0. ... 0. 0. 0.]
2502 번째: [0. 0. 0. ... 0. 0. 0.]
2511 번째: [0. 0. 0. ... 0. 0. 0.]
2520 번째: [0. 0. 0. ... 0. 0. 0.]
2529 번째: [0. 0. 0. ... 0. 0. 0.]
2538 번째: [0. 0. 0. ... 0. 0. 0.]
2547 번째: [0. 0. 0. ... 0. 0. 0.]
2556 번째: [0. 0. 0. ... 0. 0. 0.]
2565 번째: [0. 0. 0. ... 0. 0. 0.]
2574 번째: [0. 0. 0. ... 0. 0. 0.]
2583 번째: [0. 0. 0. ... 0. 0. 0.]
2592 번째: [0. 0. 0. ... 0. 0. 0.]
2601 번째: [0. 0. 0. ... 0. 0. 0.]
2610 번째: [0. 0. 0. ... 0. 0. 0.]
2619 번째: [0. 0. 0. ... 0. 0. 0.]
2628 번째: [0. 0. 0. ... 0. 0. 0.]
2637 번째: [0. 0. 0. ... 0. 0. 0.]
2646 번째: [

4617 번째: [0. 0. 0. ... 0. 0. 0.]
4626 번째: [0. 0. 0. ... 0. 0. 0.]
4635 번째: [0. 0. 0. ... 0. 0. 0.]
4644 번째: [0. 0. 0. ... 0. 0. 0.]
4653 번째: [0. 0. 0. ... 0. 0. 0.]
4662 번째: [0. 0. 0. ... 0. 0. 0.]
4671 번째: [0. 0. 0. ... 0. 0. 0.]
4680 번째: [0. 0. 0. ... 0. 0. 0.]
4689 번째: [0. 0. 0. ... 0. 0. 0.]
4698 번째: [0. 0. 0. ... 0. 0. 0.]
4707 번째: [0. 0. 0. ... 0. 0. 0.]
4716 번째: [0. 0. 0. ... 0. 0. 0.]
4725 번째: [0. 0. 0. ... 0. 0. 0.]
4734 번째: [0. 0. 0. ... 0. 0. 0.]
4743 번째: [0. 0. 0. ... 0. 0. 0.]
4752 번째: [0. 0. 0. ... 0. 0. 0.]
4761 번째: [0. 0. 0. ... 0. 0. 0.]
4770 번째: [0. 0. 0. ... 0. 0. 0.]
4779 번째: [0. 0. 0. ... 0. 0. 0.]
4788 번째: [0. 0. 0. ... 0. 0. 0.]
4797 번째: [0. 0. 0. ... 0. 0. 0.]
4806 번째: [0. 0. 0. ... 0. 0. 0.]
4815 번째: [0. 0. 0. ... 0. 0. 0.]
4824 번째: [0. 0. 0. ... 0. 0. 0.]
4833 번째: [0. 0. 0. ... 0. 0. 0.]
4842 번째: [0. 0. 0. ... 0. 0. 0.]
4851 번째: [0. 0. 0. ... 0. 0. 0.]
4860 번째: [0. 0. 0. ... 0. 0. 0.]
4869 번째: [0. 0. 0. ... 0. 0. 0.]
4878 번째: [0. 0. 0. ... 0. 0. 0.]
4887 번째: [

6858 번째: [0. 0. 0. ... 0. 0. 0.]
6867 번째: [0. 0. 0. ... 0. 0. 0.]
6876 번째: [0. 0. 0. ... 0. 0. 0.]
6885 번째: [0. 0. 0. ... 0. 0. 0.]
6894 번째: [0. 0. 0. ... 0. 0. 0.]
6903 번째: [0. 0. 0. ... 0. 0. 0.]
6912 번째: [0. 0. 0. ... 0. 0. 0.]
6921 번째: [0. 0. 0. ... 0. 0. 0.]
6930 번째: [0. 0. 0. ... 0. 0. 0.]
6939 번째: [0. 0. 0. ... 0. 0. 0.]
6948 번째: [0. 0. 0. ... 0. 0. 0.]
6957 번째: [0. 0. 0. ... 0. 0. 0.]
6966 번째: [0. 0. 0. ... 0. 0. 0.]
6975 번째: [0. 0. 0. ... 0. 0. 0.]
6984 번째: [0. 0. 0. ... 0. 0. 0.]
6993 번째: [0. 0. 0. ... 0. 0. 0.]
7002 번째: [0. 0. 0. ... 0. 0. 0.]
7011 번째: [0. 0. 0. ... 0. 0. 0.]
7020 번째: [0. 0. 0. ... 0. 0. 0.]
7029 번째: [0. 0. 0. ... 0. 0. 0.]
7038 번째: [0. 0. 0. ... 0. 0. 0.]
7047 번째: [0. 0. 0. ... 0. 0. 0.]
7056 번째: [0. 0. 0. ... 0. 0. 0.]
7065 번째: [0. 0. 0. ... 0. 0. 0.]
7074 번째: [0. 0. 0. ... 0. 0. 0.]
7083 번째: [0. 0. 0. ... 0. 0. 0.]
7092 번째: [0. 0. 0. ... 0. 0. 0.]
7101 번째: [0. 0. 0. ... 0. 0. 0.]
7110 번째: [0. 0. 0. ... 0. 0. 0.]
7119 번째: [0. 0. 0. ... 0. 0. 0.]
7128 번째: [

9099 번째: [0. 0. 0. ... 0. 0. 0.]
9108 번째: [0. 0. 0. ... 0. 0. 0.]
9117 번째: [0. 0. 0. ... 0. 0. 0.]
9126 번째: [0. 0. 0. ... 0. 0. 0.]
9135 번째: [0. 0. 0. ... 0. 0. 0.]
9144 번째: [0. 0. 0. ... 0. 0. 0.]
9153 번째: [0. 0. 0. ... 0. 0. 0.]
9162 번째: [0. 0. 0. ... 0. 0. 0.]
9171 번째: [0. 0. 0. ... 0. 0. 0.]
9180 번째: [0. 0. 0. ... 0. 0. 0.]
9189 번째: [0. 0. 0. ... 0. 0. 0.]
9198 번째: [0. 0. 0. ... 0. 0. 0.]
9207 번째: [0. 0. 0. ... 0. 0. 0.]
9216 번째: [0. 0. 0. ... 0. 0. 0.]
9225 번째: [0. 0. 0. ... 0. 0. 0.]
9234 번째: [0. 0. 0. ... 0. 0. 0.]
9243 번째: [0. 0. 0. ... 0. 0. 0.]
9252 번째: [0. 0. 0. ... 0. 0. 0.]
9261 번째: [0. 0. 0. ... 0. 0. 0.]
9270 번째: [0. 0. 0. ... 0. 0. 0.]
9279 번째: [0. 0. 0. ... 0. 0. 0.]
9288 번째: [0. 0. 0. ... 0. 0. 0.]
9297 번째: [0. 0. 0. ... 0. 0. 0.]
9306 번째: [0. 0. 0. ... 0. 0. 0.]
9315 번째: [0. 0. 0. ... 0. 0. 0.]
9324 번째: [0. 0. 0. ... 0. 0. 0.]
9333 번째: [0. 0. 0. ... 0. 0. 0.]
9342 번째: [0. 0. 0. ... 0. 0. 0.]
9351 번째: [0. 0. 0. ... 0. 0. 0.]
9360 번째: [0. 0. 0. ... 0. 0. 0.]
9369 번째: [

In [86]:
# chroma_stft 피쳐링 거친 결과를 저장하기
#np.savetxt('choma_data.csv',chroma_data)
np.savetxt('z_mfcc_data.csv',mfcc_data)

In [3]:
# chroma_stft csv를 로드한다.
#chroma_data=np.loadtxt('C:/Users/stu/chroma_data.csv')
mfcc_data=np.loadtxt("C:/Users/stu/z_mfcc_data.csv")

In [80]:
# train, test 데이터 분리
#train_set, test_set = train_test_split(chroma_data, test_size=0.3)
train_set, test_set = train_test_split(mfcc_data, test_size=0.3)
trainData = train_set[:,1:]
testData = test_set[:,1:]
trainLabel=train_set[:,0]
testLabel=test_set[:,0]
trainData.shape

(6631, 10350)

In [74]:
trainLabel.shape
testLabel.shape

(2842,)

In [81]:
## Convolution layer
tf.reset_default_graph()
# hyper parameters
learning_rate = 0.0001
training_epochs = 100
batch_size = 300
steps_for_validate = 5

X = tf.placeholder(tf.float32,[None, 10350])
X_feature = tf.reshape(X,[-1,30,345,1])
Y = tf.placeholder(tf.int32, [None, 1], name="Y")
Y_onehot=tf.reshape(tf.one_hot(Y, 41), [-1, 41])
p_keep_conv = tf.placeholder(tf.float32, name="p_keep_conv")
p_keep_hidden = tf.placeholder(tf.float32, name="p_keep_hidden")

# L1 SoundIn shape=(?, 40, 259, 1)
W1 = tf.get_variable("W1", shape=[3, 3, 1, 32],initializer=tf.contrib.layers.xavier_initializer())
L1 = tf.nn.conv2d(X_feature, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.elu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 3, 3, 1],strides=[1, 3, 3, 1], padding='SAME') 
L1 = tf.nn.dropout(L1, p_keep_conv)

# L2 Input shape=(?,10,21,32)
W2 = tf.get_variable("W2", shape=[3, 3, 32, 64],initializer=tf.contrib.layers.xavier_initializer())
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.elu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 5, 5, 1],strides=[1, 5, 5, 1], padding='SAME') 
L2 = tf.nn.dropout(L2, p_keep_conv)

# L3 Input shape=(?,3,12,64)
W3 = tf.get_variable("W3", shape=[3, 3, 64, 128],initializer=tf.contrib.layers.xavier_initializer())
L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
L3 = tf.nn.elu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1, 2, 23, 1],strides=[1, 2, 23, 1], padding='SAME') 
L3 = tf.nn.dropout(L3, p_keep_conv)
L3_flat= tf.reshape(L3, shape=[-1, 128])

# Final FC 2*3*128 inputs -> 41 outputs
W4 = tf.get_variable("W4", shape=[128, 512],initializer=tf.contrib.layers.xavier_initializer())
L4 = tf.nn.elu(tf.matmul(L3_flat, W4))
L4 = tf.nn.dropout(L4, p_keep_hidden)
W_o = tf.get_variable("W_o", shape=[512,41],initializer=tf.contrib.layers.xavier_initializer())
b = tf.Variable(tf.random_normal([41]))
logits = tf.matmul(L4, W_o) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels= Y_onehot))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam optimizer
predict_op = tf.argmax(logits, 1, name="pred")


In [82]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(trainData) / batch_size)
    for i in range(total_batch):
        batch_xs = trainData[i*batch_size:(i+1)*batch_size]
        batch_ys = trainLabel[i*batch_size:(i+1)*batch_size].reshape(-1, 1)
        feed_dict = {X: batch_xs, Y: batch_ys, p_keep_conv: .7, p_keep_hidden: 0.5}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    if epoch % steps_for_validate == steps_for_validate-1:
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y_onehot, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        x=np.random.choice(testLabel.shape[0], 500, replace=False)
        print('Accuracy:', sess.run(accuracy, feed_dict={
                X: testData[x], Y: testLabel[x].reshape(-1, 1), p_keep_conv: 1, p_keep_hidden: 1}))
        save_path = saver.save(sess, 'c:/dnn_data')
print('Finished!')

Learning started. It takes sometime.


KeyboardInterrupt: 

In [85]:
tf.__version__


'1.2.1'